In [1]:
#importing packages
library(lubridate)
library(dplyr)
library(tidyr)


Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date


Attaching package: 'dplyr'

The following objects are masked from 'package:lubridate':

    intersect, setdiff, union

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [2]:
# importing data
realized_consumption=read.csv2(file = 'GercekZamanliTuketim.csv', sep=",",stringsAsFactors = F)

# Preprocessing

In [3]:
# initializing values
cons_start_date = "01.01.2016"
cons_test_start_date = "01.11.2019"
cons_end_date = "28.11.2019" # end date is selected as one day before homework submission

In [4]:
# generating days in train and test period
itemizeDates <- function(startDate=startDate, endDate=endDate, 
                         format="%d.%m.%Y") 
{
  out <- seq(as.Date(startDate, format=format), 
             as.Date(endDate, format=format), by="days")  
  format(out, format)
}

valid_days = itemizeDates(startDate = cons_start_date, endDate = cons_end_date)

In [5]:
# checking if there are any NA values in data
any(is.na(realized_consumption[,3]))
# There are no NA values

[1] FALSE

In [6]:
# checking if there are realized consumption values for each valid days

missing_days = data.frame()

j = 1
for (i in valid_days)
{
    if(paste(i) %in% realized_consumption$Tarih == FALSE)
    {
        print(paste0("Consumption Value in Day ", paste(i), " is missing."))
        missing_days[j,"days"] = as.character(i)
        j = j + 1
    }
}



[1] "Consumption Value in Day 27.03.2016 is missing."


There is only one missing value. There are three options here. 
--> First 3 months can be discarded from the data and training is done on the remaining months (Can lead to major changes in the model, considerable number of data is discarded)
--> Code can be modified to overlook the data for 27.03.2016 (Complicated, does not add much to the model)
--> Missing 27.03.2016 value can be filled with realized consumption value of 168 hours ago (does not effect much on the model, easier to implement)
So the value is filled

In [7]:
# replacing missing date values with the 7 days prior values

# function for changing date format to char ( time YYYY-MM-DD --> char DD.MM.YYYY )
convert_time_to_char_. <- function(date) 
{
    prev_date_char = as.character(date)
    year = substr(date, 1, 4)
    month = substr(date, 6, 7)
    day = substr(date, 9, 10)
    prev_date_form = paste (day,month,year, sep = ".")
} 

# function for changing char format to date ( char DD.MM.YYYY --> time YYYY-MM-DD )
convert_date <- function(date, format_in, format_out) 
{
    temp_date <- strptime(as.character(date), format_in) 
    date = as.Date(format(temp_date, format_out))
}

# missing dates are added from 7 days ago
for(i in 1:nrow(missing_days))
{
    miss_date = convert_date(missing_days[i,"days"],"%d.%m.%Y","%Y-%m-%d")
    prev_date = miss_date - 7    
    prev_date_char = convert_time_to_char_.(prev_date)
    prev_cons=realized_consumption[realized_consumption$Tarih == prev_date_char,]
    miss_date_char = convert_time_to_char_.(miss_date)
    prev_cons[,"Tarih"] = miss_date_char   
#   realized consumption with missing values added, complete real consumption    
    cons = rbind(realized_consumption,prev_cons)
    print(head(cons))
    str(cons[1,"Tarih"])
}

       Tarih  Saat Tüketim.Miktarı..MWh.
1 01.01.2016 00:00             26.277,24
2 01.01.2016 01:00             24.991,82
3 01.01.2016 02:00             23.532,61
4 01.01.2016 03:00             22.464,78
5 01.01.2016 04:00             22.002,91
6 01.01.2016 05:00             21.957,08
 chr "01.01.2016"


In [8]:
# revising the data format and columns

# converting the date format
cons$Date = convert_date(cons$Tarih,"%d.%m.%Y","%Y-%m-%d")
# seperating hour
cons = separate(cons, Saat, into = c("Hour", "Minute"), sep = ":", remove = TRUE)
cons$Hour = as.numeric(cons$Hour) 
# making consumption as numeric
cons[,4]=as.numeric(gsub(",",".",gsub(".", "", cons[,4],fixed =T),fixed=T)) 
# finding weekday of the date
cons$Day <- wday(mdy(format(cons$Date, "%m-%d-%Y")), week_start = 1) 
# filling 48H and 168H Lag
cons['48HLag']=0
cons['168HLag']=0
cons=cons[order(cons$Date,cons$Hour, decreasing = F),] #sorting by Date and Hour
i=49
for(i in 49:nrow(cons))
{
    cons[i,7]=as.numeric(cons[i-48,4])
}
i=169
for(i in 169:nrow(cons))
{
    cons[i,8]=as.numeric(cons[i-168,4])
}
# removing unnecessary rows
cons = cons[,-3]
cons = cons[,-1]
head(cons)

Hour,Tüketim.Miktarı..MWh.,Date,Day,48HLag,168HLag
0,26277.24,2016-01-01,5,0,0
1,24991.82,2016-01-01,5,0,0
2,23532.61,2016-01-01,5,0,0
3,22464.78,2016-01-01,5,0,0
4,22002.91,2016-01-01,5,0,0
5,21957.08,2016-01-01,5,0,0


In [9]:
colnames(cons) = c("Hour", "Consumption","Date", "Day", "Lag48H", "Lag168H")
print(head(cons,100))

    Hour Consumption       Date Day   Lag48H Lag168H
1      0    26277.24 2016-01-01   5     0.00       0
2      1    24991.82 2016-01-01   5     0.00       0
3      2    23532.61 2016-01-01   5     0.00       0
4      3    22464.78 2016-01-01   5     0.00       0
5      4    22002.91 2016-01-01   5     0.00       0
6      5    21957.08 2016-01-01   5     0.00       0
7      6    22203.54 2016-01-01   5     0.00       0
8      7    21844.16 2016-01-01   5     0.00       0
9      8    23094.73 2016-01-01   5     0.00       0
10     9    25202.27 2016-01-01   5     0.00       0
11    10    27224.96 2016-01-01   5     0.00       0
12    11    28908.04 2016-01-01   5     0.00       0
13    12    28789.25 2016-01-01   5     0.00       0
14    13    29367.70 2016-01-01   5     0.00       0
15    14    29548.32 2016-01-01   5     0.00       0
16    15    29390.89 2016-01-01   5     0.00       0
17    16    30734.97 2016-01-01   5     0.00       0
18    17    32048.02 2016-01-01   5     0.00  

# Task a

In [25]:
# Creating test data
naive_test_cons = cons[cons$Date >= convert_date(cons_test_start_date,"%d.%m.%Y","%Y-%m-%d") & 
                       cons$Date <= convert_date(cons_end_date,"%d.%m.%Y","%Y-%m-%d"),]

print(naive_test_cons)
# Building Naive Forecast

# Calculating MAPE values
naive_test_cons$Mape_Lag48 = round((abs(naive_test_cons$Consumption - naive_test_cons$Lag48H) / naive_test_cons$Consumption * 100), 2)
naive_test_cons$Mape_Lag168 = round((abs(naive_test_cons$Consumption - naive_test_cons$Lag168H) / naive_test_cons$Consumption * 100), 2)


      Hour Consumption       Date Day   Lag48H  Lag168H
33577    0    29417.56 2019-11-01   5 27154.21 29563.43
33578    1    28133.75 2019-11-01   5 26157.42 28242.90
33579    2    27358.60 2019-11-01   5 25373.88 27258.74
33580    3    26780.09 2019-11-01   5 24911.43 26739.84
33581    4    26511.54 2019-11-01   5 24836.11 26555.35
33582    5    27002.74 2019-11-01   5 25233.76 26857.36
33583    6    27945.43 2019-11-01   5 26296.00 27783.77
33584    7    29120.27 2019-11-01   5 27575.60 28969.45
33585    8    32815.46 2019-11-01   5 31667.27 32153.21
33586    9    34569.09 2019-11-01   5 33138.17 33615.22
33587   10    35091.43 2019-11-01   5 32926.25 33398.50
33588   11    35416.33 2019-11-01   5 33122.35 33542.88
33589   12    33184.81 2019-11-01   5 31518.65 30839.72
33590   13    33549.94 2019-11-01   5 31895.21 30920.91
33591   14    35732.88 2019-11-01   5 33050.83 33019.99
33592   15    35859.75 2019-11-01   5 33464.69 33476.62
33593   16    36268.51 2019-11-01   5 34612.24 3

In [26]:
# Preparing the Table for MAPE Values of both Naive Models
naive_model = c("Lag48", "Lag168")
naive_MAPE = c(paste(round(mean(naive_test_cons$Mape_Lag48),2),"%"),paste(round(mean(naive_test_cons$Mape_Lag168),2),"%")) 
naive_standard_dev = c(paste(round(sd(naive_test_cons$Mape_Lag48),2),"%"),paste(round(sd(naive_test_cons$Mape_Lag168),2),"%"))

naive_MAPE_table = data.frame(naive_model, naive_MAPE, naive_standard_dev)
print(naive_MAPE_table)

  naive_model naive_MAPE naive_standard_dev
1       Lag48     7.71 %             8.38 %
2      Lag168     3.06 %             3.33 %


MAPE and Standard Deviation of Lag168 is much lower. That means, it explains variance better. So Lag168 is preferred.